# # Project: NLP to Analyze Sentiments of Consumer Reviews of Hotels    

In this project, we use Sentiment Analysis of NLP to analyze reviews of hotels. We would like to understand if a review is positive or negative (without looking at the ratings). We will use the following libraries:
- NLTK: the most famous python module for NLP techniques
- Gensim: a topic-modelling and vector space modelling toolkit
- Scikit-learn: the most used python machine learning library

The dataset is from Kaggle and contains 515,000 customer reviews and scoring of 1493 luxury hotels offered in Booking.come across Europe.
The dataset can be found here: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe/version/1

In [4]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Part 1: Load the Data and Take a Look

In [5]:
# load the data into datadrames
businesses = pd.read_json('/Users/Amir/PythonProjects/YelpDataset/yelp_business.json', lines=True)
reviews = pd.read_json('/Users/Amir/PythonProjects/YelpDataset/yelp_review.json', lines=True)
users = pd.read_json('/Users/Amir/PythonProjects/YelpDataset/yelp_user.json', lines=True)
checkins = pd.read_json('/Users/Amir/PythonProjects/YelpDataset/yelp_checkin.json', lines=True)
tips = pd.read_json('/Users/Amir/PythonProjects/YelpDataset/yelp_tip.json', lines=True)
photos = pd.read_json('/Users/Amir/PythonProjects/YelpDataset/yelp_photo.json', lines=True)
# Set `max_columns` to `60` and `max_colwidth` to `500`. We are working with some BIG data here!
pd.options.display.max_columns = 60
pd.options.display.max_colwidth = 500

In [6]:
businesses.head(3)

,address,alcohol?,attributes,business_id,categories,city,good_for_kids,has_bike_parking,has_wifi,hours,is_open,latitude,longitude,name,neighborhood,postal_code,price_range,review_count,stars,state,take_reservations,takes_credit_cards
0,1314 44 Avenue NE,0,"{'BikeParking': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'GoodForKids': 'True', 'HasTV': 'True', 'NoiseLevel': 'average', 'OutdoorSeating': 'False', 'RestaurantsAttire': 'casual', 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'True', 'RestaurantsTakeOut': 'True'}",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel",Calgary,1,0,0,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21:0', 'Thursday': '11:0-21:0', 'Friday': '11:0-21:0', 'Saturday': '11:0-21:0'}",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,2,24,4.0,AB,1,1
1,,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}', 'Caters': 'True', 'DogsAllowed': 'True', 'DriveThru': 'False', 'GoodForKids': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'breakfast': False, 'brunch': False}', 'HasTV': 'False', 'OutdoorSeating': 'True', 'RestaurantsAttire': 'casual', 'RestaurantsDelivery'...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services",Henderson,1,0,0,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0', 'Sunday': '17:0-23:0'}",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,2,3,4.5,NV,0,1
2,1335 rue Beaubien E,1,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}', 'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'Caters': 'False', 'GoodForKids': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'breakfa...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandwiches, Cafes",Montréal,1,1,1,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0', 'Wednesday': '10:0-22:0', 'Thursday': '10:0-22:0', 'Friday': '10:0-22:0', 'Saturday': '10:0-22:0', 'Sunday': '10:0-22:0'}",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,2,5,4.0,QC,1,0


In [7]:
businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 22 columns):
address               188593 non-null object
alcohol?              188593 non-null int64
attributes            162807 non-null object
business_id           188593 non-null object
categories            188052 non-null object
city                  188593 non-null object
good_for_kids         188593 non-null int64
has_bike_parking      188593 non-null int64
has_wifi              188593 non-null int64
hours                 143791 non-null object
is_open               188593 non-null int64
latitude              188587 non-null float64
longitude             188587 non-null float64
name                  188593 non-null object
neighborhood          188593 non-null object
postal_code           188593 non-null object
price_range           188593 non-null int64
review_count          188593 non-null int64
stars                 188593 non-null float64
state                 188593 non-null 

In [8]:
# Example: what is the Yelp rating, or `stars`, of the establishment with `business_id` = `5EvUIR4IzCWUOm0PsUZXjA`
businesses[businesses['business_id'] == '5EvUIR4IzCWUOm0PsUZXjA']['stars']

30781    3.0
Name: stars, dtype: float64

## Part 2: 

## Part 3: Clean the Data

In [1]:
# Drop unwante columns and return results in the same dataframe
#features_to_remove = ['address','attributes','business_id','categories','city','hours','is_open','latitude','longitude','name','neighborhood','postal_code','state','time']
# axis=1 lets Pandas know we want to drop columns, not rows
# inplace=True lets us drop the columns right here in our DataFrame, instead of returning a new DataFrame
#df.drop(labels=features_to_remove, axis=1, inplace=True)
#df.info()

In [3]:
# check for missing values (if any, they will prevent LR from running)
#df.isna().any()
# replace all of our NaNs with 0s
#df.fillna({'weekday_checkins':0,'weekend_checkins':0, 'average_tip_length':0, 'number_tips':0, 'average_caption_length':0,'number_pics':0}, inplace=True)
# check for missing values
#df.isna().any()

## Part 4: 

## Part 5: 

## Part 6: 